In [1]:
#DQN model for BTC closing price prediction

__author__ = 'Prof.D'


In [2]:
# Importing core libraries
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

import tensorflow as tf
from tensorflow import keras

style.use('seaborn-whitegrid')
np.random.seed(42)


/home/student/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Importing dataset
df = pd.read_csv(r'coindesk-bpi-USD-close_data-2010-07-17_2018-08-01.csv')
df.head()


,Date,Close Price
0,7/18/2010 0:00,0.09
1,7/19/2010 0:00,0.08
2,7/20/2010 0:00,0.07
3,7/21/2010 0:00,0.08
4,7/22/2010 0:00,0.05


In [4]:
# Simple numeric encoding of 'Date' variable.
#Starts at day 0, ends at day 2937. Date == index.
df['Date'] = np.arange(len(df['Date']))
print(df['Date'].head(), df['Date'].tail())


0    0
1    1
2    2
3    3
4    4
Name: Date, dtype: int64 2933    2933
2934    2934
2935    2935
2936    2936
2937    2937
Name: Date, dtype: int64


In [5]:
# Features and labels
X = df['Date'].values
y = df['Close Price'].values 


In [6]:
# Splitting into training and testing sets
#No validation set due to small size of data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [7]:
#Defining DQN Agent class
import random
from collections import deque
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam

#Class based on code from: https://github.com/llSourcell/Reinforcement_Learning_for_Stock_Prediction/blob/master/agent/agent.py
class Agent:
    def __init__(self, state_size, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995, eta=0.001, is_eval=False, model_name=""):
        self.state_size = state_size
        self.action_size = 3 # sit, buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval

        self.gamma = gamma #0.95
        self.epsilon = epsilon #1.0
        self.epsilon_min = epsilon_min #0.01
        self.epsilon_decay = epsilon_decay #0.995
        self.eta = eta #0.001
        
        self.model = load_model("models/" + model_name) if is_eval else self._model()

    def _model(self, n_nodes_l1=64, n_nodes_l2=32, n_nodes_l3=8):
        model = Sequential()
        model.add(Dense(units=n_nodes_l1, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=n_nodes_l2, activation="relu"))
        model.add(Dense(units=n_nodes_l3, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(lr=self.eta))

        return model

    def act(self, state):
        if not self.is_eval and np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)

        options = self.model.predict(state)
        return np.argmax(options[0])

    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])

        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay 
            

Using TensorFlow backend.


In [ ]:
#Helper functions
def getState(data, t, n):
    d = t - n + 1
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
    res = []
    for i in range(n - 1):
        res.append(sigmoid(block[i + 1] - block[i]))
    return np.array([res])

def formatPrice(n):
    return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

def sigmoid(x):
    return (1 / ( 1 + np.exp(-x) ) )

# Initializing and training agent
window_size, episode_count = 10, 1000

agent = Agent(window_size)
data = list(y_train) #Close price values
l = len(data) - 1
batch_size = 32

for e in range(episode_count + 1):
    print("Episode " + str(e) + "/" + str(episode_count))
    state = getState(data, 0, window_size + 1)

    total_profit = 0
    agent.inventory = []
    
    for t in range(l):
        action = agent.act(state)
        
        # sit
        next_state = getState(data, t + 1, window_size + 1)
        reward = 0

        if action == 1: # buy
            agent.inventory.append(data[t])
            print("Buy: " + formatPrice(data[t]))

        elif action == 2 and len(agent.inventory) > 0: # sell
            bought_price = agent.inventory.pop(0)
            reward = max(data[t] - bought_price, 0)
            total_profit += data[t] - bought_price
            print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
        
        done = True if t == l - 1 else False
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state
        
        if done:
            print('------------------------------------')
            print("Total Profit: " + formatPrice(total_profit))
            print('------------------------------------')
        
        if len(agent.memory) > batch_size:
            agent.expReplay(batch_size)
        
    if e % 10 == 0:
        #print('Saving...')
        agent.model.save('models/model_ep' + str(e))
        

Episode 0/1000
Buy: $11158.39
Sell: $1639.32 | Profit: -$9519.07
Buy: $5.35
Buy: $1037.75
Buy: $385.86
Sell: $5.91 | Profit: $0.56
Buy: $236.10
Buy: $0.06
Buy: $572.67
Sell: $234.21 | Profit: -$803.54
Buy: $8.50
Buy: $2499.98
Sell: $96.92 | Profit: -$288.94
Sell: $6.80 | Profit: -$229.30
Sell: $380.04 | Profit: $379.98
Buy: $421.20
Sell: $1179.97 | Profit: $607.30
Sell: $3.25 | Profit: -$5.25
Buy: $2.56
Buy: $3431.97
Sell: $1938.94 | Profit: -$561.04
Buy: $12.68
Buy: $459.46
Sell: $239.83 | Profit: -$181.37
Sell: $2873.83 | Profit: $2871.27
Buy: $0.27
Buy: $246.13
Sell: $6.67 | Profit: -$3425.30
Sell: $29.89 | Profit: $17.21
Sell: $649.81 | Profit: $190.35
Buy: $4.39
Buy: $0.39
Sell: $0.83 | Profit: $0.56
Buy: $5.45
Sell: $473.83 | Profit: $227.70
Sell: $1042.03 | Profit: $1037.64
Buy: $257.66
Buy: $2.33
Sell: $474.13 | Profit: $473.74
Sell: $5.12 | Profit: -$0.33
Buy: $239.29
Buy: $11.73
Sell: $85.78 | Profit: -$171.88
Sell: $232.12 | Profit: $229.79
Buy: $814.53
Sell: $654.98 | Profi

Sell: $137.42 | Profit: -$107.91
Sell: $138.64 | Profit: -$879.16
Sell: $221.85 | Profit: $208.55
Sell: $951.39 | Profit: $937.86
Sell: $0.09 | Profit: -$8215.47
Sell: $750.63 | Profit: -$3026.66
Sell: $672.48 | Profit: -$6357.52
Sell: $0.06 | Profit: -$7.74
Sell: $271.92 | Profit: $271.22
Buy: $225.39
Buy: $6.76
Buy: $264.96
Buy: $5.21
Buy: $2197.23
Buy: $6.80
Buy: $1.08
Buy: $8891.21
Buy: $8229.96
Buy: $6664.21
Buy: $238.59
Buy: $13.60
Buy: $380.08
Buy: $15.70
Sell: $398.43 | Profit: $173.04
Sell: $363.31 | Profit: $356.55
Sell: $226.50 | Profit: -$38.46
Sell: $8183.02 | Profit: $8177.81
Sell: $274.41 | Profit: -$1922.82
Sell: $5.10 | Profit: -$1.70
Sell: $625.49 | Profit: $624.41
Sell: $11.80 | Profit: -$8879.41
Sell: $4.01 | Profit: -$8225.95
Sell: $561.35 | Profit: -$6102.86
Sell: $199.62 | Profit: -$38.97
Sell: $450.99 | Profit: $437.39
Sell: $607.37 | Profit: $227.29
Sell: $3678.74 | Profit: $3663.04
Buy: $5984.09
Sell: $500.26 | Profit: -$5483.83
Buy: $873.26
Sell: $241.19 | Pr

Sell: $3.15 | Profit: -$2.44
Sell: $997.69 | Profit: $994.66
Sell: $8.20 | Profit: $2.63
Sell: $485.83 | Profit: $479.14
Sell: $1263.54 | Profit: -$12925.24
Sell: $999.51 | Profit: $988.64
Sell: $124.19 | Profit: $7.37
Sell: $810.58 | Profit: $796.44
Buy: $633.18
Buy: $444.25
Buy: $653.93
Buy: $13585.90
Buy: $6247.47
Buy: $415.38
Buy: $4.86
Buy: $7689.91
Buy: $13.37
Buy: $2591.22
Buy: $848.29
Buy: $4137.67
Buy: $1.04
Sell: $1191.81 | Profit: $558.63
Sell: $12.20 | Profit: -$432.05
Sell: $598.60 | Profit: -$55.33
Sell: $575.73 | Profit: -$13010.17
Sell: $10313.08 | Profit: $4065.61
Sell: $5984.96 | Profit: $5569.58
Sell: $1.11 | Profit: -$3.75
Sell: $0.06 | Profit: -$7689.85
Sell: $295.83 | Profit: $282.46
Sell: $74.22 | Profit: -$2517.00
Sell: $10.96 | Profit: -$837.33
Sell: $4.27 | Profit: -$4133.40
Sell: $229.76 | Profit: $228.72
Buy: $0.70
Buy: $122.50
Buy: $4.87
Buy: $3.52
Buy: $1244.37
Buy: $6.11
Buy: $66.34
Buy: $0.98
Buy: $2.29
Buy: $9362.53
Buy: $11.89
Sell: $0.06 | Profit: -$0

Buy: $638.37
Buy: $11228.24
Buy: $0.15
Buy: $87.46
Buy: $4.85
Buy: $64.49
Buy: $10.94
Buy: $912.69
Buy: $333.55
Buy: $427.24
Buy: $125.45
Buy: $16937.17
Buy: $4.62
Buy: $9279.00
Buy: $93.00
Buy: $16699.68
Buy: $766.75
Buy: $387.66
Buy: $0.75
Buy: $125.84
Buy: $14.78
Buy: $1.07
Buy: $702.50
Buy: $230.20
Sell: $103.33 | Profit: $102.28
Buy: $7049.79
Sell: $232.97 | Profit: -$184.04
Sell: $0.06 | Profit: -$1017.99
Sell: $227.33 | Profit: -$4537.54
Sell: $2051.73 | Profit: $2044.87
Sell: $5.49 | Profit: $0.88
Sell: $6758.38 | Profit: $6753.57
Sell: $0.90 | Profit: -$1.00
Sell: $635.51 | Profit: $634.61
Sell: $386.59 | Profit: -$217.99
Sell: $12.25 | Profit: -$1.24
Sell: $623.26 | Profit: $623.16
Sell: $237.31 | Profit: -$8437.90
Sell: $13619.03 | Profit: $13464.83
Sell: $11.67 | Profit: $8.90
Sell: $6.35 | Profit: -$4153.11
Sell: $4.84 | Profit: $0.15
Sell: $0.06 | Profit: -$207.57
Sell: $9.35 | Profit: -$267.27
Sell: $11.77 | Profit: -$2727.16
Sell: $10456.17 | Profit: $6030.87
Sell: $12.

Sell: $7404.29 | Profit: $7403.41
Sell: $573.55 | Profit: $564.75
Sell: $659.69 | Profit: $652.27
Sell: $2579.93 | Profit: $2203.89
Sell: $229.74 | Profit: $216.32
Sell: $8138.34 | Profit: $8028.61
Sell: $630.15 | Profit: $467.85
Sell: $430.84 | Profit: $409.66
Sell: $420.33 | Profit: -$253.26
Buy: $1009.25
Buy: $6.31
Buy: $225.75
Sell: $11616.85 | Profit: $10607.60
Sell: $16.90 | Profit: $10.59
Sell: $627.91 | Profit: $402.16
Buy: $246.85
Buy: $5.28
Buy: $107.55
Buy: $0.07
Buy: $5.03
Buy: $281.38
Buy: $449.86
Buy: $2.33
Buy: $746.72
Buy: $676.91
Buy: $711.96
Sell: $8.41 | Profit: -$238.44
Sell: $4408.32 | Profit: $4403.04
Sell: $0.06 | Profit: -$107.49
Sell: $4.87 | Profit: $4.80
Sell: $125.51 | Profit: $120.48
Sell: $577.86 | Profit: $296.48
Sell: $4.92 | Profit: -$444.94
Sell: $679.73 | Profit: $677.40
Sell: $429.72 | Profit: -$317.00
Sell: $6504.93 | Profit: $5828.02
Sell: $2814.36 | Profit: $2102.40
Buy: $0.05
Buy: $27.10
Buy: $11.81
Buy: $8.85
Buy: $14.24
Buy: $3.58
Buy: $0.39
Bu

...

Sell: $240.22 | Profit: $230.56
Sell: $5.12 | Profit: -$224.21
Buy: $2.05
Sell: $284.50 | Profit: $282.45
Buy: $222.23
Buy: $0.23
Sell: $374.73 | Profit: $152.50
Sell: $455.31 | Profit: $455.08
Buy: $326.00
Buy: $0.52
Buy: $742.01
Buy: $3.64
Buy: $9.81
Buy: $10.89
Buy: $0.08
Buy: $768.24
Buy: $471.36
Buy: $264.87
Buy: $117.45
Buy: $8938.30
Sell: $263.63 | Profit: -$62.37
Buy: $429.46
Buy: $4.96
Buy: $5.18
Buy: $765.21
Buy: $8.74
Buy: $5.10
Buy: $591.03
Buy: $583.11
Buy: $6.42
Buy: $871.05
Buy: $235.30
Buy: $47.00
Sell: $6517.68 | Profit: $6517.16
Sell: $7776.94 | Profit: $7034.93
Sell: $598.41 | Profit: $594.77
Sell: $444.81 | Profit: $435.00
Sell: $581.83 | Profit: $570.94
Sell: $571.69 | Profit: $571.61
Sell: $513.94 | Profit: -$254.30
Sell: $399.48 | Profit: -$71.88
Sell: $422.30 | Profit: $157.43
Sell: $216.77 | Profit: $99.32
Sell: $437.51 | Profit: -$8500.79
Sell: $8234.15 | Profit: $7804.69
Sell: $7616.10 | Profit: $7611.14
Sell: $207.22 | Profit: $202.04
Sell: $120.74 | Profit:

Sell: $843.76 | Profit: -$2609.40
Buy: $129.30
Buy: $17.51
Buy: $0.06
Buy: $0.20
Sell: $84.61 | Profit: $79.64
Sell: $120.57 | Profit: -$8.73
Sell: $758.81 | Profit: $741.30
Buy: $293.79
Buy: $230.05
Buy: $86.94
Buy: $395.86
Buy: $3.95
Buy: $591.97
Sell: $2332.19 | Profit: $2332.13
Buy: $4.89
Buy: $496.62
Sell: $5.01 | Profit: $4.81
Sell: $232.79 | Profit: -$61.00
Buy: $433.59
Sell: $601.71 | Profit: $371.66
Buy: $13.50
Buy: $685.99
Sell: $564.37 | Profit: $477.43
Buy: $0.06
Sell: $234.68 | Profit: -$161.18
Sell: $7396.29 | Profit: $7392.34
Sell: $12.37 | Profit: -$579.60
Sell: $676.11 | Profit: $671.22
Sell: $457.85 | Profit: -$38.77
Sell: $106.25 | Profit: -$327.34
Sell: $9692.72 | Profit: $9679.22
Sell: $0.30 | Profit: -$685.69
Sell: $0.85 | Profit: $0.79
Buy: $4.27
Sell: $460.70 | Profit: $456.43
Buy: $4.33
Buy: $768.62
Sell: $8095.59 | Profit: $8091.26
Sell: $2525.67 | Profit: $1757.05
Buy: $2.97
Buy: $3.20
Buy: $8.18
Buy: $0.06
Sell: $12.50 | Profit: $9.53
Sell: $567.20 | Profit:

Sell: $775.98 | Profit: $770.87
Sell: $0.30 | Profit: -$3.59
Sell: $0.06 | Profit: -$574.39
Sell: $4.84 | Profit: -$393.52
Sell: $4.25 | Profit: -$632.53
Sell: $411.42 | Profit: $325.33
Sell: $222.84 | Profit: -$232.91
Sell: $2452.18 | Profit: $2197.37
Sell: $6.65 | Profit: -$1248.75
Sell: $10035.00 | Profit: $10030.21
Sell: $666.18 | Profit: $663.15
Sell: $362.73 | Profit: $99.83
Sell: $4.84 | Profit: -$698.85
Sell: $778.68 | Profit: $773.90
Buy: $444.32
Buy: $0.07
Sell: $434.46 | Profit: -$9.86
Sell: $611.78 | Profit: $611.71
Buy: $1.14
Sell: $969.62 | Profit: $968.48
Buy: $4.36
Sell: $235.81 | Profit: $231.45
Buy: $15.44
Sell: $8763.78 | Profit: $8748.34
Buy: $6570.31
Buy: $15.40
Buy: $1147.25
Buy: $121.29
Buy: $372.38
Buy: $373.32
Sell: $4191.22 | Profit: -$2379.09
Sell: $1178.38 | Profit: $1162.98
Sell: $326.08 | Profit: -$821.17
Sell: $4322.75 | Profit: $4201.46
Sell: $1054.34 | Profit: $681.96
Sell: $244.32 | Profit: -$129.00
Buy: $6.63
Sell: $990.00 | Profit: $983.37
Buy: $421.

Buy: $19.84
Sell: $694.47 | Profit: $689.55
Buy: $3.04
Sell: $665.33 | Profit: $374.82
Buy: $265.37
Buy: $90.54
Sell: $111.44 | Profit: -$317.61
Sell: $853.02 | Profit: $197.89
------------------------------------
Total Profit: $270299.63
------------------------------------
Episode 56/1000
Buy: $11158.39
Buy: $1639.32
Buy: $5.35
Buy: $1037.75
Buy: $385.86
Buy: $5.91
Buy: $236.10
Buy: $0.06
Buy: $424.40
Sell: $617.82 | Profit: -$10540.57
Sell: $9.99 | Profit: -$1629.33
Sell: $448.88 | Profit: $443.53
Sell: $474.13 | Profit: -$563.62
Sell: $348.96 | Profit: -$36.90
Sell: $452.11 | Profit: $446.20
Buy: $178.12
Sell: $5809.69 | Profit: $5573.59
Sell: $633.67 | Profit: $633.61
Sell: $5.22 | Profit: -$419.18
Sell: $731.75 | Profit: $553.63
Buy: $1.00
Sell: $268.11 | Profit: $267.11
Buy: $0.32
Sell: $446.96 | Profit: $446.64
Buy: $253.69
Buy: $194.55
Buy: $8240.72
Buy: $3445.28
Buy: $106.35
Buy: $592.26
Sell: $223.31 | Profit: -$30.38
Sell: $415.16 | Profit: $220.61
Sell: $371.25 | Profit: -

Buy: $12.45
Buy: $0.19
Sell: $510.43 | Profit: $497.98
Sell: $585.08 | Profit: $584.89
Buy: $0.30
Buy: $0.19
Sell: $234.25 | Profit: $233.95
Sell: $463.74 | Profit: $463.55
Buy: $573.55
Buy: $659.69
Sell: $2579.93 | Profit: $2006.38
Buy: $229.74
Buy: $8138.34
Buy: $630.15
Buy: $430.84
Buy: $420.33
Buy: $6.65
Buy: $0.06
Buy: $436.61
Sell: $1018.11 | Profit: $358.42
Sell: $2247.48 | Profit: $2017.74
Buy: $6830.90
Buy: $3.08
Buy: $602.89
Buy: $408.80
Buy: $359.35
Buy: $988.95
Buy: $455.73
Buy: $1512.21
Buy: $0.11
Sell: $631.39 | Profit: -$7506.95
Sell: $8051.34 | Profit: $7421.19
Sell: $12.40 | Profit: -$418.44
Sell: $17608.35 | Profit: $17188.02
Sell: $245.26 | Profit: $238.61
Sell: $1216.50 | Profit: $1216.44
Sell: $386.27 | Profit: -$50.34
Sell: $334.97 | Profit: -$6495.93
Sell: $1053.12 | Profit: $1050.04
Sell: $0.23 | Profit: -$602.66
Sell: $238.08 | Profit: -$170.72
Sell: $323.26 | Profit: -$36.09
Sell: $242.64 | Profit: -$746.31
Sell: $328.98 | Profit: -$126.75
Sell: $5.56 | Profit

In [ ]:
# Loading and evaluating model performance
from keras.models import load_model

def evaluate(data, data_name='BTC test set', model_episode_num=1000, batch_size=32):
    #data_name = 'Bitcoin test set'
    #episode_num = 1000 #Final training episode
    model_name = 'model_ep{}'.format(model_episode_num) #Model from episode {episode_num}
    model = load_model("models/" + model_name)
    window_size = model.layers[0].input.shape.as_list()[1]

    agent = Agent(window_size, True, model_name)
    data = list(data)
    l = len(data) - 1
    #batch_size = 32

    state = getState(data, 0, window_size+1)
    total_profit = 0
    agent.inventory = []

    for t in range(l):
        action = agent.act(state)

        #sit
        next_state = getState(data, t+1, window_size+1)
        reward = 0

        #buy
        if action == 1:
            agent.inventory.append(data[t])
            print('Buy: ' + formatPrice(data[t]))

        #sell
        elif action == 2 and len(agent.inventory) > 0:
            bought_price = agent.inventory.pop(0)
            reward = max(data[t] - bought_price, 0)
            total_profit += data[t] - bought_price
            print('Sell: ' + formatPrice(data[t]) + '| Profit: ' + formatPrice(data[t] - bought_price))

        done = True if t == l - 1 else False
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state

        if done:
            print('------------------------------------')
            print(data_name + ' Total Profit: ' + formatPrice(total_profit))
            print('------------------------------------')
            return formatPrice(total_profit)

total_profit_test_set = evaluate(y_test)
        

In [ ]:
# Debugging cell
'''
# This function just extracts the value at adj close in the originial code
def getStockDataVec(key):
    vec = []
    lines = open("./" + key + ".csv", "r").read().splitlines()
    lines.remove(lines[1:][-1])
    
    for i,line in enumerate(lines[1:]):
        #print(lines[1:][:2])
        vec.append(float(line.split(',')[6]))
#       print(i)
        #vec.append(float(line.split(",")[4]))

    return vec

example_data = getStockDataVec('GSPC')
data = y_train
print(np.array(example_data).shape, data.shape)

#example_state = getState(example_data, 0, window_size + 1)
#state = getState(data, 0, window_size + 1)
t = 0
n = window_size+1
d = t - n + 1
#data = example_data
data = list(y_train) #Turns out indexing error was because y_train was a np.array instead of a list like the data in the original code.
block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
# print(len(block), d, data[0:t + 1], len(data[0:t + 1]))
print(len(-d * [data[0]] + data[0:t+1]))
'''